In [ ]:
import great_expectations as ge
from wpi_onderzoekswaardigheid_aanvraag.project_paths import ARTIFACT_PATH, DATA_PATH, PACKAGE_ROOT
from wpi_onderzoekswaardigheid_aanvraag.expectations.check_expectations import load_expectations_from_path

In [ ]:
file_path = DATA_PATH / 'transformed_data.pkl.gz'
df = ge.read_pickle(file_path)

In [ ]:
# Load existing expectations
EXPECTATIONS_PATH = str(PACKAGE_ROOT / "expectations/expectations_pipeline_output.json")
es = load_expectations_from_path(EXPECTATIONS_PATH)
df._expectation_suite = es

In [ ]:
for c in [
    "application_dienstnr",
    "dtaanvraag",
    "subjectnr",
]:
    df.expect_column_values_to_not_be_null(c, meta={"train_only": False})
df.expect_column_values_to_not_be_null("onderzoekswaardig", meta={"train_only": True})
df.expect_column_values_to_be_unique("application_dienstnr", meta={"train_only": False})
df.expect_column_mean_to_be_between("onderzoekswaardig", 0.01, 0.04, meta={"train_only": True})
df.expect_column_mean_to_be_between("afgewezen", 0.35, 0.45, meta={"train_only": True})
df.expect_column_distinct_values_to_be_in_set("srp_id", [
    734,
    735,
    736,
    737,
    738,
    739,
    760,
    780,
    781,
    800,
    945,
    946,
    947,
    948,
    949,
    1048,
    1089,
    1090,
    1091,
    1124,
    1125,
    1126,
], meta={"train_only": True})
df.expect_column_distinct_values_to_be_in_set("application_productnr", [
    131,  # WWb/LO
    135,  # WWb/EV
    87,   # IOAW
    88,   # IOAZ
    227,  # Krediethypotheek
], meta={"train_only": False})

In [ ]:
es = df.get_expectation_suite(discard_failed_expectations=False)
print(es)

In [ ]:
# Save expectation suite (note: this will overwrite the existing expectations!)
df.save_expectation_suite(EXPECTATIONS_PATH)